load libraries

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

from keras.models import *
from keras.optimizers import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.callbacks import *
from keras.layers import *

import itertools
import keras

Using TensorFlow backend.


In [2]:
dir = "/ext/Data/aichallenger/scene/"

model_image_size = (299, 299)
batch_size = 32

classdf = pd.read_csv("scene_classes.csv")
for i in range(80):
    if i % 10 == 9:
        print(classdf.loc[i]["chinese"])

橄榄球场
拳击场
塔
会议室
商店
舞厅
沟渠
雪屋/冰雕(山)


# 加载数据集

load train data

In [3]:
train_gen = ImageDataGenerator(
#     featurewise_std_normalization=True,
#    samplewise_std_normalization=False,
#     rotation_range=10.,
#     width_shift_range=0.05,
#     height_shift_range=0.05,
#     shear_range=0.1,
#     zoom_range=0.1,
)
gen = ImageDataGenerator(
#     featurewise_std_normalization=True,
#     samplewise_std_normalization=False,
)

classes = []
for i in range(80):
    classes.append(str(i))
train_generator = train_gen.flow_from_directory(os.path.join(dir, 'train'),  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical", classes=classes)
print("subdior to train type {}".format(train_generator.class_indices))
valid_generator = gen.flow_from_directory(os.path.join(dir, 'valid'),  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical", classes=classes)
print("subdior to valid type {}".format(valid_generator.class_indices))

Found 107758 images belonging to 80 classes.
subdior to train type {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '21': 21, '22': 22, '23': 23, '24': 24, '25': 25, '26': 26, '27': 27, '28': 28, '29': 29, '30': 30, '31': 31, '32': 32, '33': 33, '34': 34, '35': 35, '36': 36, '37': 37, '38': 38, '39': 39, '40': 40, '41': 41, '42': 42, '43': 43, '44': 44, '45': 45, '46': 46, '47': 47, '48': 48, '49': 49, '50': 50, '51': 51, '52': 52, '53': 53, '54': 54, '55': 55, '56': 56, '57': 57, '58': 58, '59': 59, '60': 60, '61': 61, '62': 62, '63': 63, '64': 64, '65': 65, '66': 66, '67': 67, '68': 68, '69': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79}
Found 7120 images belonging to 80 classes.
subdior to valid type {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 

# 构建模型

In [4]:
def make_model(optimizer, dropout, lr, tune_layer_resnet50, tune_layer_inceptionV3, tune_layer_xception):
    input_tensor = Input((*model_image_size, 3))
    x = input_tensor
    inception_v3_x = Lambda(inception_v3.preprocess_input)(x)
    xception_x = Lambda(xception.preprocess_input)(x)

    resnet50_model = ResNet50(input_tensor=x, weights='imagenet', include_top=False)
    resnet50_model_output = GlobalAveragePooling2D()(resnet50_model.output)
    for i in range(tune_layer_resnet50):
        resnet50_model.layers[i].trainable = False
        
    inceptionV3_model = InceptionV3(input_tensor=inception_v3_x, weights='imagenet', include_top=False)
    inceptionV3_model_output = GlobalAveragePooling2D()(inceptionV3_model.output)
    for i in range(tune_layer_inceptionV3):
        inceptionV3_model.layers[i].trainable = False
    
        
    xception_model = Xception(input_tensor=xception_x, weights='imagenet', include_top=False)
    xception_model_output = GlobalAveragePooling2D()(xception_model.output)
    for i in range(tune_layer_xception):
        xception_model.layers[i].trainable = False
    
    x= Concatenate(axis=-1)([resnet50_model_output,  inceptionV3_model_output,  xception_model_output])
    #x= Concatenate(axis=-1)([inceptionV3_model_output, xception_model_output])
    #x = inceptionV3_model_output
    x = Dropout(dropout)(x)
    x = Dense(80, activation='softmax')(x)
    model = Model(inputs=input_tensor, outputs=x)
    
    print("total layer count {}".format(len(model.layers)))
        
    if optimizer == "adam":
        optimizer_class = Adam(lr=lr)
    elif optimizer == "rmsprop":
        optimizer_class = RMSprop(lr=lr)
        
    model.compile(optimizer=optimizer_class, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 训练模型

In [5]:
print("train_generator.samples = {}".format(train_generator.samples))
print("valid_generator.samples = {}".format(valid_generator.samples))
steps_train_sample = train_generator.samples // batch_size + 1
steps_valid_sample = valid_generator.samples // batch_size + 1

train_generator.samples = 107758
valid_generator.samples = 7120


In [ ]:
if not os.path.exists("models/mixed"):
    os.mkdir("models/mixed")

In [ ]:
import itertools

optimizers = ['adam']
dropouts = [0.5]
lrs = [0.001,  0.0001]
resnet50_tune_layers = [172, 162,  140]
inceptionV3_tune_layers = [253,  213,  173]
xception_tune_layers = [126, 116,  96]

parameters = itertools.product(optimizers, dropouts, lrs, resnet50_tune_layers, inceptionV3_tune_layers, xception_tune_layers)    

for p in parameters:
    name = "optimizer,{}-dropout,{}-lr,{}-tune_layer,({},{},{})".format(p[0],p[1],p[2],p[3],p[4],p[5])
    final_filepath = "models/mixed/" + name + ".h5"
    best_filepath="models/mixed/" + name + "__{epoch:03d}- Loss:{val_loss:.3f}.h5"
    print()
    print(name)
    callbacks = [EarlyStopping(monitor='val_loss',patience=6),ModelCheckpoint(best_filepath, monitor='val_loss',save_best_only=True)] 
    model = make_model(p[0],p[1],p[2],p[3],p[4],p[5])
    model.fit_generator(train_generator, 
                                                  steps_per_epoch=steps_train_sample, 
                                                  validation_data=valid_generator, 
                                                  validation_steps=steps_valid_sample, 
                                                  epochs=500,
                                                  callbacks=callbacks)
    model.save(final_filepath)
print("model saved!")


optimizer,adam-dropout,0.5-lr,0.001-tune_layer,(172,253,126)
total layer count 623
Epoch 1/500
3368/3368 [==============================] - 2374s - loss: 1.4456 - acc: 0.6452 - val_loss: 1.1775 - val_acc: 0.7221

# 可视化模型

https://keras.io/visualization/

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import *

model = load_model("models/mixed/002- Loss:1.027.h5")
print("load successed")

#SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
z = zip([x.name for x in model.layers], range(len(model.layers)))
for k, v in z:
    print("{} - {}".format(k,v))

http://docs.opencv.org/trunk/d3/d50/group__imgproc__colormap.html

![](http://docs.opencv.org/trunk/colorscale_jet.jpg)

In [ ]:
import json

def gen_test_result(model,  model_image_size, json_name):
    gen = ImageDataGenerator()
    test_generator = gen.flow_from_directory(dir + "test/",  model_image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)

    y_pred = model.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)
    print("y_pred shape {}".format(y_pred.shape))
    print(y_pred[0])

    l = list()
    for i, fname in enumerate(test_generator.filenames):
        name = fname[fname.rfind('/')+1:]
        d = dict()
        d["image_id"] = name
        d["label_id"] = y_pred[i].argsort()[-3:][::-1].tolist()
        l.append(d)
   
    json.dump(l, open(json_name, "w"))
    print("json saved")

print("done")

In [ ]:
gen_test_result(model,  model_image_size, 'json/mixed-imagenet-finetune-pred.json')